In [1]:
pip install pandas


Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install mysql-connector-python

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install matplotlib

Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install seaborn

Note: you may need to restart the kernel to use updated packages.


In [5]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns


In [2]:
import os
os.getcwd()


'C:\\JupyterProjects'

In [4]:
os.listdir()


['.ipynb_checkpoints',
 'Customer_Shopping_Behaviour_Analysis.ipynb',
 'Ecommerce',
 'ecommerce.ipynb']

In [5]:
import os

os.chdir(r"C:\Users\Dell\customer-trends-data-analysis-SQL-Python-PowerBI")


In [6]:
os.getcwd()
os.listdir()


['.git',
 'Business Problem  Document.pdf',
 'Customer Shopping Behavior Analysis.pdf',
 'Customer-Shopping-Behavior-Analysis.pptx',
 'customer_behavior_dashboard.pbix',
 'customer_behavior_sql_queries.sql',
 'customer_shopping_behavior.csv',
 'Customer_Shopping_Behavior_Analysis.ipynb',
 'LICENSE',
 'README.md']

In [7]:
import pandas as pd

df = pd.read_csv("customer_shopping_behavior.csv")
df.head()


,Customer ID,Age,Gender,Item Purchased,Category,Purchase Amount (USD),Location,Size,Color,Season,Review Rating,Subscription Status,Shipping Type,Discount Applied,Promo Code Used,Previous Purchases,Payment Method,Frequency of Purchases
0,1,55,Male,Blouse,Clothing,53,Kentucky,L,Gray,Winter,3.1,Yes,Express,Yes,Yes,14,Venmo,Fortnightly
1,2,19,Male,Sweater,Clothing,64,Maine,L,Maroon,Winter,3.1,Yes,Express,Yes,Yes,2,Cash,Fortnightly
2,3,50,Male,Jeans,Clothing,73,Massachusetts,S,Maroon,Spring,3.1,Yes,Free Shipping,Yes,Yes,23,Credit Card,Weekly
3,4,21,Male,Sandals,Footwear,90,Rhode Island,M,Maroon,Spring,3.5,Yes,Next Day Air,Yes,Yes,49,PayPal,Weekly
4,5,45,Male,Blouse,Clothing,49,Oregon,M,Turquoise,Spring,2.7,Yes,Free Shipping,Yes,Yes,31,PayPal,Annually


In [9]:
import mysql.connector
import numpy as np

# Connect to MySQL
conn = mysql.connector.connect(
    host="localhost",
    user="root",
    password="butch@777",
    database="customer_db"
)
cursor = conn.cursor()

# Drop table safely (once)
cursor.execute("DROP TABLE IF EXISTS customer_shopping_behavior")

# Create table dynamically
def mysql_type(dtype):
    if "int" in str(dtype):
        return "INT"
    elif "float" in str(dtype):
        return "FLOAT"
    else:
        return "TEXT"

columns_sql = ", ".join(
    [f"`{col}` {mysql_type(df[col].dtype)}" for col in df.columns]
)

create_table_sql = f"""
CREATE TABLE customer_shopping_behavior (
    {columns_sql}
)
"""

cursor.execute(create_table_sql)

# Insert data (FAST)
placeholders = ", ".join(["%s"] * len(df.columns))
insert_sql = f"""
INSERT INTO customer_shopping_behavior ({", ".join(df.columns)})
VALUES ({placeholders})
"""

data = [tuple(None if pd.isna(x) else x for x in row) for row in df.values]
cursor.executemany(insert_sql, data)

conn.commit()
conn.close()

print("✅ Data loaded successfully into MySQL!")
print("Rows inserted:", len(data))


✅ Data loaded successfully into MySQL!
Rows inserted: 3900


In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3900 entries, 0 to 3899
Data columns (total 18 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   customer_id             3900 non-null   int64  
 1   age                     3900 non-null   int64  
 2   gender                  3900 non-null   object 
 3   item_purchased          3900 non-null   object 
 4   category                3900 non-null   object 
 5   purchase_amount_usd     3900 non-null   int64  
 6   location                3900 non-null   object 
 7   size                    3900 non-null   object 
 8   color                   3900 non-null   object 
 9   season                  3900 non-null   object 
 10  review_rating           3863 non-null   float64
 11  subscription_status     3900 non-null   object 
 12  shipping_type           3900 non-null   object 
 13  discount_applied        3900 non-null   object 
 14  promo_code_used         3900 non-null   

In [11]:
df.columns = (
    df.columns
      .str.lower()
      .str.strip()
      .str.replace(" ", "_")
      .str.replace("-", "_")
      .str.replace("(", "")
      .str.replace(")", "")
)

df.columns


Index(['customer_id', 'age', 'gender', 'item_purchased', 'category',
       'purchase_amount_usd', 'location', 'size', 'color', 'season',
       'review_rating', 'subscription_status', 'shipping_type',
       'discount_applied', 'promo_code_used', 'previous_purchases',
       'payment_method', 'frequency_of_purchases'],
      dtype='object')

In [12]:
df.describe(include = 'all')

,customer_id,age,gender,item_purchased,category,purchase_amount_usd,location,size,color,season,review_rating,subscription_status,shipping_type,discount_applied,promo_code_used,previous_purchases,payment_method,frequency_of_purchases
count,3900.000000,3900.000000,3900,3900,3900,3900.000000,3900,3900,3900,3900,3863.000000,3900,3900,3900,3900,3900.000000,3900,3900
unique,NaN,NaN,2,25,4,NaN,50,4,25,4,NaN,2,6,2,2,NaN,6,7
top,NaN,NaN,Male,Blouse,Clothing,NaN,Montana,M,Olive,Spring,NaN,No,Free Shipping,No,No,NaN,PayPal,Every 3 Months
freq,NaN,NaN,2652,171,1737,NaN,96,1755,177,999,NaN,2847,675,2223,2223,NaN,677,584
mean,1950.500000,44.068462,NaN,NaN,NaN,59.764359,NaN,NaN,NaN,NaN,3.750065,NaN,NaN,NaN,NaN,25.351538,NaN,NaN
std,1125.977353,15.207589,NaN,NaN,NaN,23.685392,NaN,NaN,NaN,NaN,0.716983,NaN,NaN,NaN,NaN,14.447125,NaN,NaN
min,1.000000,18.000000,NaN,NaN,NaN,20.000000,NaN,NaN,NaN,NaN,2.500000,NaN,NaN,NaN,NaN,1.000000,NaN,NaN
25%,975.750000,31.000000,NaN,NaN,NaN,39.000000,NaN,NaN,NaN,NaN,3.100000,NaN,NaN,NaN,NaN,13.000000,NaN,NaN
50%,1950.500000,44.000000,NaN,NaN,NaN,60.000000,NaN,NaN,NaN,NaN,3.800000,NaN,NaN,NaN,NaN,25.000000,NaN,NaN
75%,2925.250000,57.000000,NaN,NaN,NaN,81.000000,NaN,NaN,NaN,NaN,4.400000,NaN,NaN,NaN,NaN,38.000000,NaN,NaN


In [13]:
df.isnull().sum()

Customer ID                0
Age                        0
Gender                     0
Item Purchased             0
Category                   0
Purchase Amount (USD)      0
Location                   0
Size                       0
Color                      0
Season                     0
Review Rating             37
Subscription Status        0
Shipping Type              0
Discount Applied           0
Promo Code Used            0
Previous Purchases         0
Payment Method             0
Frequency of Purchases     0
dtype: int64

In [ ]:
# How should we replace the null values - 
By mean and median now generally we use median over mean
because mean can generally affected by outliers and that would skew the distribution of our data 
whereas median is not affected by outliers
We will impute the missing values using the median review rating within each category and will replace that with the missing value. 
That way if a cothing item is missing its review rating, it gets replaced with the median review rating of a clothing item not a footwear item

In [20]:
df['review_rating'] = (
    df.groupby('category')['review_rating']
      .transform(lambda x: x.fillna(x.median()))
)

df.isnull().sum()


customer_id               0
age                       0
gender                    0
item_purchased            0
category                  0
purchase_amount_usd       0
location                  0
size                      0
color                     0
season                    0
review_rating             0
subscription_status       0
shipping_type             0
discount_applied          0
promo_code_used           0
previous_purchases        0
payment_method            0
frequency_of_purchases    0
dtype: int64

In [25]:
print(df.columns.tolist())


['customer_id', 'age', 'gender', 'item_purchased', 'category', 'purchase_amount_usd', 'location', 'size', 'color', 'season', 'review_rating', 'subscription_status', 'shipping_type', 'discount_applied', 'promo_code_used', 'previous_purchases', 'payment_method', 'frequency_of_purchases']


In [28]:
df.columns = df.columns.str.lower().str.replace(" ", "_")


In [29]:
df.columns

Index(['customer_id', 'age', 'gender', 'item_purchased', 'category',
       'purchase_amount_usd', 'location', 'size', 'color', 'season',
       'review_rating', 'subscription_status', 'shipping_type',
       'discount_applied', 'promo_code_used', 'previous_purchases',
       'payment_method', 'frequency_of_purchases'],
      dtype='object')

In [ ]:
# Although the steps were the same as the reference, the initial dataframe state differed
due to hidden execution history. After normalizing column names correctly, the results
aligned with the expected output.


In [18]:
# Create a column age_group
labels = ['Young Adult', 'Adult', 'Middle-Aged', 'Senior']
df['age_group'] = pd.qcut(df['age'], q = 4, labels = labels)

In [19]:
df[['age','age_group']].head(10)

,age,age_group
0,55,Middle-Aged
1,19,Young Adult
2,50,Middle-Aged
3,21,Young Adult
4,45,Middle-Aged
5,46,Middle-Aged
6,63,Senior
7,27,Young Adult
8,26,Young Adult
9,57,Middle-Aged


In [20]:
# create column purchase_frequency_days

frequency_mapping = {
    'Fortnightly': 14,
    'Weekly': 7,
    'Monthly': 30,
    'Quarterly': 90,
    'Bi-weekly': 14,
    'Annually': 365,
    'Every 3 Months': 90
}

df['purchase_frequency_days'] = df['frequency_of_purchases'].map(frequency_mapping)
    


In [ ]:
First we created a dictionary called frequency mapping that maps each textual frequency
to the equivalent numberof days.
Lets say fortnightly is 14 days, week is 7 days and so on.Then we use a map function to
replace the text in the column with the corresponding number of days

In [21]:
df[['purchase_frequency_days','frequency_of_purchases']].head(10)

,purchase_frequency_days,frequency_of_purchases
0,14.0,Fortnightly
1,14.0,Fortnightly
2,7.0,Weekly
3,7.0,Weekly
4,365.0,Annually
5,7.0,Weekly
6,90.0,Quarterly
7,7.0,Weekly
8,365.0,Annually
9,90.0,Quarterly


In [22]:
df[['discount_applied','promo_code_used']].head(10)

,discount_applied,promo_code_used
0,Yes,Yes
1,Yes,Yes
2,Yes,Yes
3,Yes,Yes
4,Yes,Yes
5,Yes,Yes
6,Yes,Yes
7,Yes,Yes
8,Yes,Yes
9,Yes,Yes


In [ ]:
# Do we actually need both of these columns? Do both of these columns 
have the same values? Is one of the columns redundant?

In [23]:
(df['discount_applied'] == df['promo_code_used']).all()

np.True_

In [ ]:
# Both columns contained identical values across all rows,
so one was redundant and removed to avoid multicollinearity and reduce noise.

In [24]:
df = df.drop('promo_code_used', axis = 1)

In [26]:
pip install mysql-connector-python sqlalchemy


Note: you may need to restart the kernel to use updated packages.


In [ ]:
-- Q1. What is the total revenue generated by male vs. female customers?

SELECT gender, SUM(purchase_amount) AS revenue
FROM customer_shopping_behavior
GROUP BY gender;

-- Q2. Which customers used a discount but still spent more than the average purchase amount?


select cutomer_id, purchase_amount 
from customer_shopping_behavior
where discount_applied = 'Yes' and purchase_amount >= (select AVG(purchase_amount) from customer)

-- Q3. Which are the top 5 products with the highest average revenue ratings?
-- Q4. Compare the average purchase amounts between Standard and Express Shipping.
-- Q5. Do subscribed customers spend more? Compare average spend and total revenue between subscribers and non-subscribers.
-- Q6. Which 5 products have the highest percentage of purchases with discounts applied?
-- Q7. Segment customers into New, Returning, and Loyal based on their total number of previous purchases, and show the count of each segment.
-- Q8. What are the top 3 most purchased products within each category?
-- Q9. Are customers who are repeat buyers (more than 5 previous purchases) also likely to subscribe?
-- Q10. What is the revenue contribution of each age group?

In [32]:
pip install pymysql sqlalchemy


Note: you may need to restart the kernel to use updated packages.


In [34]:
from sqlalchemy import create_engine

# MySQL connection
username = "root"
password = "butch%40777"   # @ replaced with %40
host = "localhost"
port = "3306"
database = "customer_db"

engine = create_engine(
    f"mysql+pymysql://{username}:{password}@{host}:{port}/{database}"
)

